In [17]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop

In [30]:
train = ImageDataGenerator(rescale=1/255)
validation = ImageDataGenerator(rescale=1/255)
testing = ImageDataGenerator(rescale=1/255)

In [57]:
train_dataset = train.flow_from_directory("/content/drive/MyDrive/tf_workspace/training_cnn/images/training",target_size=(200,200),batch_size=8)
validation_dataset = validation.flow_from_directory("/content/drive/MyDrive/tf_workspace/training_cnn/images/validation",target_size=(200,200),batch_size=8)
testing_dataset = testing.flow_from_directory("/content/drive/MyDrive/tf_workspace/training_cnn/images/testing",target_size=(200,200),batch_size=8, shuffle=False)

Found 4129 images belonging to 4 classes.
Found 486 images belonging to 4 classes.
Found 241 images belonging to 4 classes.


In [20]:
train_dataset.class_indices

{'bee': 0, 'bumbee': 1, 'insect': 2, 'vespula': 3}

In [21]:
def convolutional_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same', strides = (2,2))(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv2D(filter, (1,1), strides = (2,2))(x_skip)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x
def identity_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x
def ResNet34(shape = (32, 32, 3), classes = 10):
    # Step 1 (Setup Input Layer)
    x_input = tf.keras.layers.Input(shape)
    x = tf.keras.layers.ZeroPadding2D((3, 3))(x_input)
    # Step 2 (Initial Conv layer along with maxPool)
    x = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    # Define size of sub-blocks and initial filter size
    block_layers = [3, 4, 6, 3]
    filter_size = 64
    # Step 3 Add the Resnet Blocks
    for i in range(4):
        if i == 0:
            # For sub-block 1 Residual/Convolutional block not needed
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            # One Residual/Convolutional Block followed by Identity blocks
            # The filter size will go on increasing by a factor of 2
            filter_size = filter_size*2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)
    # Step 4 End Dense Network
    x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation = 'relu')(x)
    x = tf.keras.layers.Dense(classes, activation = 'softmax')(x)
    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
    return model
def customModel():
  return tf.keras.models.Sequential([tf.keras.layers.Conv2D(32,(3,3),activation = "relu", input_shape =(200,200,3)),
                              #Preprocessing
                              tf.keras.layers.RandomContrast(factor=0.1, seed=None),
                              tf.keras.layers.RandomRotation(factor=1,fill_mode='reflect',interpolation='bilinear',seed=None,fill_value=0.0),
                              tf.keras.layers.RandomFlip(),
                              #Training
                              tf.keras.layers.MaxPool2D(3,3),
                              tf.keras.layers.Conv2D(32,(3,3),activation = "relu"),
                              tf.keras.layers.MaxPool2D(3,3),
                              tf.keras.layers.Conv2D(64,(3,3),activation = "relu"),
                              tf.keras.layers.MaxPool2D(3,3),
                              tf.keras.layers.Conv2D(128,(3,3),activation = "relu"),
                              tf.keras.layers.MaxPool2D(3,3),
                              tf.keras.layers.Flatten(),
                              tf.keras.layers.Dense(512, activation = "relu"),
                              tf.keras.layers.Dense(4, activation = "softmax"),
                              ])

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')
tf.device('/device:GPU:0')
model = tf.keras.models.Sequential([
    # Input layer
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3), padding='same'),
    
    # Preprocessing
    tf.keras.layers.RandomRotation(factor=1, fill_mode='reflect', interpolation='bilinear', seed=None, fill_value=0.0),
    tf.keras.layers.RandomFlip(),
    
    # Feature extraction
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
    # Classifier
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4, activation='softmax')
])
print(type(model))

<class 'keras.engine.sequential.Sequential'>


In [22]:
model =ResNet34(shape = (200, 200, 3), classes = 4)
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "/content/drive/MyDrive/tf_workspace/CNN_Test_5000_ResNet/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
batch_size = 8
# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=100*3)

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))
model.compile(optimizer='adam', metrics=["accuracy"], loss=tf.keras.losses.CategoricalCrossentropy())

In [23]:
with tf.device('/device:GPU:0'):
  model_fit = model.fit(train_dataset,steps_per_epoch = 3, batch_size = batch_size, callbacks=[cp_callback], epochs = 5000, validation_data=validation_dataset)

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
3/3 [==============================] - 2s 824ms/step - loss: 0.2420 - accuracy: 0.9583 - val_loss: 0.7016 - val_accuracy: 0.7449
Epoch 2527/5000
3/3 [==============================] - 2s 821ms/step - loss: 0.2904 - accuracy: 0.8750 - val_loss: 0.7088 - val_accuracy: 0.7490
Epoch 2528/5000
3/3 [==============================] - 2s 807ms/step - loss: 0.2229 - accuracy: 0.8750 - val_loss: 0.7488 - val_accuracy: 0.7469
Epoch 2529/5000
3/3 [==============================] - 2s 1s/step - loss: 0.3947 - accuracy: 0.7917 - val_loss: 0.8234 - val_accuracy: 0.7346
Epoch 2530/5000
3/3 [==============================] - 2s 968ms/step - loss: 0.1481 - accuracy: 0.9583 - val_loss: 0.9343 - val_accuracy: 0.7202
Epoch 2531/5000
3/3 [==============================] - 2s 821ms/step - loss: 0.2526 - accuracy: 0.9167 - val_loss: 1.0178 - val_accuracy: 0.6975
Epoch 2532/5000
3/3 [==============================] - 2s 822ms/step - loss: 0.353

In [24]:
validation_dataset.class_indices

{'bee': 0, 'bumbee': 1, 'insect': 2, 'vespula': 3}

In [ ]:
from tensorflow.keras.preprocessing import image
from skimage import transform
validation_dataset.class_indices
dir_path = "/content/drive/MyDrive/tf_workspace/training_cnn/images/validation"
detWrong =[]
classes = 4
detWrongElement = []
for count in range(classes):
  detWrongElement.append([0,0,0,0,0])
for i in os.listdir(dir_path):
  for files in os.listdir(os.path.join(dir_path, i)):
    active_dir = os.path.join(dir_path, i)
    imagePath = os.path.join(active_dir, files)
    img = image.load_img(imagePath, target_size=(200,200))
    X = image.img_to_array(img)
    X = X.reshape(1, 200, 200, 3)
    #X = np.expand_dims(X,axis=0)
    #images = np.vstack([X])
    images = X.astype("float32")
    val = model.predict(images)
    detectionList = val.tolist()
    filename, _ = os.path.splitext(files)
    fileType = filename.split('(')
    fileTypeValue = None
    if(fileType[0] == "bee"):
      fileTypeValue = 0
    if(fileType[0] == "bumbeBee" or fileType[0] == "bumbleBee"):
      fileTypeValue = 1
    if(fileType[0] == "insect"):
      fileTypeValue = 2
    if(fileType[0] == "vespula"):
      fileTypeValue = 3
    if(fileTypeValue != None and detectionList[0][fileTypeValue] < 0.8):
      detWrong.append([detectionList[0],files])
      detWrongElement[fileTypeValue][0] +=1
      for index, element in enumerate(detectionList[0]):
        if(element >= 0.8):
          print(f"{filename} was identified as {index} with value {detectionList[0][fileTypeValue]}")
          print(detectionList)
          detWrongElement[fileTypeValue][index+1] +=1
    elif(fileTypeValue == None):
      print(filename)
print("bee, bumblebee, insect, vespula")
print(detWrongElement)

In [ ]:
# Validation set evaluation
val_loss, val_accuracy = model.evaluate(testing_dataset,
                                        steps=int(val_size/bs)+1)
print('Accuracy: {}'.format(val_accuracy))

# Validation set predict

y_val = testing_dataset.classes

pred = model.predict(testing_dataset,
                     verbose=1
                     steps=int(val_size/bs)+1)

accuracy_TTA = np.mean(np.equal(y_val, np.argmax(pred, axis=-1)))
print('Accuracy: {}'.format(accuracy_TTA))

In [73]:
y_val = testing_dataset.classes
pred = model.predict(testing_dataset)
accuracy_TTA = np.mean(np.equal(y_val, np.argmax(pred, axis=-1)))
print('Accuracy: {}'.format(accuracy_TTA))

31/31 [==============================] - 2s 52ms/step
[[9.56498444e-01 2.67096126e-04 4.32254821e-02 9.00720624e-06]
 [2.80750811e-01 9.61833848e-07 7.18619049e-01 6.29206595e-04]
 [9.99999166e-01 7.62520145e-08 7.62993352e-07 2.03222078e-10]
 [8.61336067e-02 8.24014563e-03 9.05597448e-01 2.88614428e-05]
 [1.81258712e-02 4.09190194e-04 9.80528772e-01 9.36163007e-04]
 [6.52802612e-07 9.99977112e-01 2.22605595e-05 3.54087391e-08]
 [9.99892712e-01 7.17558214e-05 3.55131087e-05 2.12887352e-09]
 [9.98290718e-01 1.56932184e-03 1.39953729e-04 2.60888672e-11]
 [3.80434457e-08 5.09334750e-05 9.99948978e-01 9.50595602e-09]
 [9.95892763e-01 4.08690143e-03 2.02411229e-05 1.96807406e-08]
 [1.35470688e-01 1.22586347e-03 8.50862384e-01 1.24410633e-02]
 [9.99687314e-01 5.68288783e-07 1.21920406e-04 1.90150706e-04]
 [9.99997616e-01 6.54230137e-09 2.13648139e-08 2.32955608e-06]
 [9.65727985e-01 1.99787226e-03 3.00521497e-02 2.22201529e-03]
 [9.99997020e-01 5.45236123e-09 2.99293356e-06 1.08773796e-10]
 

In [58]:
print(model.evaluate(testing_dataset, batch_size=128))

31/31 [==============================] - 1s 32ms/step - loss: 1.1707 - accuracy: 0.7552
[1.1707438230514526, 0.7551867365837097]


In [77]:
from PIL import Image
import numpy as np
from skimage import transform
def load(filename):
   np_image = Image.open(filename)
   np_image = np.array(np_image).astype('float32')/255
   np_image = transform.resize(np_image, (200, 200, 3))
   np_image = np.expand_dims(np_image, axis=0)
   return np_image

image = load('/content/drive/MyDrive/tf_workspace/training_cnn/images/validation/bee/bee(1082).jpg')
print(model.predict(image))

1/1 [==============================] - 0s 158ms/step
[[9.922937e-01 7.231133e-05 7.633988e-03 5.142945e-09]]


In [81]:
from tensorflow.keras.preprocessing import image
from PIL import Image
import numpy as np
from skimage import transform
def load(filename):
   np_image = Image.open(filename)
   np_image = np.array(np_image).astype('float32')/255
   np_image = transform.resize(np_image, (200, 200, 3))
   np_image = np.expand_dims(np_image, axis=0)
   return np_image
validation_dataset.class_indices
dir_path = "/content/drive/MyDrive/tf_workspace/training_cnn/images/validation"
detWrong =[]
classes = 4
detWrongAmount = 0
detRightAmount = 0
testedImages = 0
detWrongElement = []
for count in range(classes):
  detWrongElement.append([0,0,0,0,0])
for i in os.listdir(dir_path):
  for files in os.listdir(os.path.join(dir_path, i)):
    testedImages += 1
    active_dir = os.path.join(dir_path, i)
    imagePath = os.path.join(active_dir, files)
    images = load(imagePath)
    val = model.predict(images)
    detectionList = val.tolist()
    filename, _ = os.path.splitext(files)
    fileType = filename.split('(')
    fileTypeValue = None
    if(fileType[0] == "bee"):
      fileTypeValue = 0
    if(fileType[0] == "bumbeBee" or fileType[0] == "bumbleBee"):
      fileTypeValue = 1
    if(fileType[0] == "insect"):
      fileTypeValue = 2
    if(fileType[0] == "vespula"):
      fileTypeValue = 3
    if(fileTypeValue != None and detectionList[0][fileTypeValue] < 0.5):
      detWrongAmount += 1
      detWrong.append([detectionList[0],files])
      detWrongElement[fileTypeValue][0] +=1
      for index, element in enumerate(detectionList[0]):
        if(element >= 0.5):
          detWrongElement[fileTypeValue][index+1] +=1
    elif(fileTypeValue != None): detRightAmount += 1
    elif(fileTypeValue == None):
      print(filename)
print("bee, bumblebee, insect, vespula")
print(detWrongElement)
print(f"{testedImages} davon {detWrongAmount} falsch und {detRightAmount} richtig. Genauigkeit ist {detRightAmount / testedImages}")

1/1 [==============================] - 0s 33ms/step
bee, bumblebee, insect, vespula
[[38, 0, 20, 18, 0], [21, 11, 0, 8, 0], [43, 25, 14, 0, 3], [6, 1, 1, 4, 0]]
486 davon 108 falsch und 378 richtig. Genauigkeit ist 0.7777777777777778


### Get number of files within the folder

In [79]:
dirPath = "/content/drive/MyDrive/tf_workspace/training_cnn/images/validation"
os.listdir(dirPath)
for elements in os.listdir(dirPath):
  print(f"{elements}: {len(os.listdir(os.path.join(dirPath, elements)))}")

vespula: 32
insect: 237
bumbee: 91
bee: 126
